In [2]:
import numpy as np, pandas as pd
from cloud_kaggle_lib import *
from IPython.display import clear_output
from tqdm import tqdm_notebook
PATH = '/Users/voanhkha/Desktop/understanding_cloud_organization/'
from tqdm import tqdm_notebook as tqdm
train_df = pd.read_csv(PATH+'train.csv')
sub_df = pd.read_csv(PATH+'sample_submission.csv')
train_df['im_id'] = train_df['Image_Label'].apply(lambda x: x.split('_')[0])
sub_df['im_id'] = sub_df['Image_Label'].apply(lambda x: x.split('_')[0])
ori_names = train_df.iloc[::4]['im_id'].values
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import precision_recall_curve, auc

In [8]:
# Simply load single model oof and pred
MODELPATH = '/Users/voanhkha/Desktop/preds_cloud/finalblend0/'
oof_name = np.load(MODELPATH + 'names.npy', allow_pickle=True)
oof_pred = np.load(MODELPATH + 'oof.npy')
test_pred = np.load(MODELPATH + 'test.npy')

In [ ]:
# Postprocess with triplets
# Remember: /100 or not

for k in range(0,4):
    print('k =', k)

    #k = 0 # layer CLOUDTYPES = ['Fish', 'Flower', 'Gravel', 'Sugar']
    gt_all = np.load('/Users/voanhkha/Desktop/understanding_cloud_organization/gt_' +str(k)+ '.npy')
    if k==2: correct_k = 3
    elif k==3: correct_k = 2
    else: correct_k = k   
        
    all_layers = np.zeros((5546, 350, 525))
    for j, (img_name, pred) in tqdm_notebook(enumerate(zip(oof_name, oof_pred))):
        pred = pred / 100
        im_layer = pred[:,:,k]
        all_layers[j] = im_layer
        
    # Second: search for params
    best = [0,0,0,0]
    for threshold in [.6, .65, .85, .8, .7, .75]:
        for minsize in [12500, 5000, 7500, 0, 2500, 10000]:
    
            for bottom_threshold in [.3, .35 , .4, .45]:
        
                score = 0
                for j, (img_name, pred) in enumerate(zip(oof_name, oof_pred)):
                    im_layer = all_layers[j]
                    gt_layer = gt_all[ori_names==img_name]
                    
                    ## Adaptive param
                    clf_prob = clf_oof[clf_names==img_name][0][correct_k]
                    adaptive_ms = minsize - (clf_prob - 0.5)*0
#                     adaptive_thres = threshold - (clf_prob - 0.5)*0
                    
                    adaptive_thres = threshold + 0.05 if clf_prob < 0.2 else threshold
                     

                    ##   
                    
                    pred_thres = cv2.threshold(im_layer, adaptive_thres, 1, cv2.THRESH_BINARY)[1]
                    
                    #### Draw convex
                    pred_thres = draw_convex_hull(pred_thres.astype(np.uint8))
                    ####
                    
                    num_component, component = cv2.connectedComponents(pred_thres.astype(np.uint8))
                    pred_mask = np.zeros((350, 525), np.uint8) 
                    
                    for c in range(1, num_component):
                        p = (component == c)
                        if p.sum() > adaptive_ms: pred_mask[p] = 1
                    
                ### Triplet section ###
                    if pred_mask.sum() > 0: pred_mask = cv2.threshold(im_layer, bottom_threshold, 1, cv2.THRESH_BINARY)[1]  
                ############################    
                    
                    score += dice(pred_mask, gt_layer)    
                        
                score = score / (len(oof_name))
                print(threshold, minsize, np.round(bottom_threshold,2) , np.round(score,5))
                if best[-1] < score: best = [threshold, minsize, bottom_threshold, score]
                    
    print('Best:', best)

In [27]:
# Make test prediction (triplet)
## REMEMBER: /100

LAYERS = [0, 1, 2, 3]
postprocess_params = {'thres': [70, 70, 70, 70], 
                      'minsize': [5000, 5000,5000, 5000], 
                      'lowerthres':[40,40,40,40]}
encoded_pixels = []
for img in tqdm(test_pred):
    img = img/100
    for k in LAYERS:
        ### REMEMBER TO CHANGE BELOW LINE 
        im_layer = img[:,:,k] # or img[j]

        thres, minsize, lowerthres = postprocess_params['thres'][k], postprocess_params['minsize'][k], postprocess_params['lowerthres'][k]
        
        pred_thres = cv2.threshold(im_layer, thres/100, 1, cv2.THRESH_BINARY)[1]
        num_component, component = cv2.connectedComponents(pred_thres.astype(np.uint8))
        pred_mask = np.zeros((350, 525), np.uint8) 
        for c in range(1, num_component):
            p = (component == c)
            if p.sum() > minsize: pred_mask[p] = 1

    ### Triplet section ###
        if pred_mask.sum() > 0: pred_mask = cv2.threshold(im_layer, lowerthres/100, 1, cv2.THRESH_BINARY)[1] 
            

        encoded_pixels.append(mask2rle(pred_mask))

In [28]:
sub = pd.read_csv(PATH+'sample_submission.csv')
sub['EncodedPixels'] = encoded_pixels
#sub[['Image_Label', 'EncodedPixels']].to_csv('/Users/voanhkha/Desktop/B0_80.gz', index=None, compression='gzip')
sub.head()

,Image_Label,EncodedPixels
0,002f507.jpg_Fish,
1,002f507.jpg_Flower,
2,002f507.jpg_Gravel,3 334 351 67187 67540 347 67891 346 68242 345 ...
3,002f507.jpg_Sugar,
4,0035ae9.jpg_Fish,
